In [1]:
import numpy as np

In [35]:
def algorithm_2(A, m, p):
    '''
    A: a symmetric positive definite matrix of size n times n for some n  
    m: the chosen number of samples
    p: the chosen probabibilty for the confidence interval
    
    return: estimator: an unbiased estimator of the quantity tr(f(A))
            (L_p, U_p): the bounds of the confidence interval of tr(f(A)) with proba p 
    '''
    
    if np.any(A!=A.T):
        raise ValueError('The matrix isnt symmetric')
        
    if np.any(np.linalg.eigvals(A) <= 0):
        raise ValueError('The matrix isnt positive definite')
        
    if p<=0 or p>=1:
        raise ValueError('we dont have 0<p<1')
        
    n=A.shape[0]
    L=np.ones(m)*10^8 #dummy value to start with, L isn't suposed to be bigger than 10^8 ever
    U=np.ones(m)*-10^8 #dummy value to start with, U isn't suppose to be smaller than -10^8 ever
    
    for j in range(m):
        z = np.random.uniform(0,1,n)
        z[z<0.5]=-1
        z[z>=0.5]=1
        #apply algorithm 1 to obtain a lower bound L_j and an upper bound U_j and set 
        #L[j]=L_j and U[j]=U_j
        
        I=(L_j+U_j).T@np.ones(j)/(2*j)
        L=np.minimum(L,L_j)
        U=np.maximum(U,U_j)
        eta_2=-0.5*j*(U-L)^2*(np.log(1-p)/2)
        temp=np.concatenate([np.ones(j),np.zeros(n-j)])
        L_p_j= temp@L/j-np.sqrt(eta_2)/j
        U_p_j= temp@U/j+np.sqrt(eta_2)/j
        
    return I, L_p_j, U_p_j     